#**Test**

In [ ]:
def record_audio(filename='output.wav', duration=3, fs=16000):
    print("Gravando...")
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()  # Espera a gravação ser concluída
    print("Gravação concluída.")

    # Salva o arquivo WAV
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)  # 2 bytes para int16
        wf.setframerate(fs)
        wf.writeframes(audio.tobytes())
    return filename

In [ ]:
def predict_command(model, audio_filename):
    mfccs = process_audio(audio_filename)
    # Adiciona uma dimensão extra para a previsão
    mfccs = np.expand_dims(mfccs, axis=0)  # Forma: (1, height, width, channels)

    prediction = model.predict(mfccs)
    predicted_label = np.argmax(prediction, axis=1)
    return predicted_label


In [ ]:
# Função para carregar o arquivo de áudio
def load_audio_file(filepath):
    # Carrega o arquivo de áudio com taxa de amostragem de 16kHz
    audio, sr = librosa.load(filepath, sr=16000)
    return audio, sr

In [ ]:
# Definindo a função para calcular os MFCCs
def get_mfccs(audio_file, sr=16000, n_mfcc=13):
    # Carregar o áudio
    audio, _ = librosa.load(audio_file, sr=sr)

    # Calcular os MFCCs
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)

    return mfccs

In [ ]:
# Caminho para o arquivo de áudio
audio_file = '/content/left.opus'

# Ajustar a função para aceitar arrays de áudio
def get_mfccs(audio, sr=16000, n_mfcc=13):
    # Calcular os MFCCs diretamente a partir do array de áudio
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    return mfccs.T  # Transpor para ter (time_steps, n_mfcc)

# Agora você pode calcular os MFCCs de um array diretamente
mfccs_teste = get_mfccs(audio)  # Passe o array de áudio diretamente


# Verifique a forma das MFCCs
print(f"Forma dos MFCCs antes do ajuste: {mfccs.shape}")

# Verifique a forma dos MFCCs
print(f"Forma dos MFCCs antes do ajuste: {mfccs_teste.shape}")

# Adicionar dimensões para batch e canais
mfccs_teste = np.expand_dims(mfccs, axis=0)  # Adiciona a dimensão do batch
mfccs_teste = np.expand_dims(mfccs, axis=-1) # Adiciona a dimensão do canal

# Exibir a nova forma dos MFCCs
print(f"Forma dos MFCCs após ajuste: {mfccs_teste.shape}")

In [ ]:
mfccs = mfccs[np.newaxis, ..., np.newaxis]  # Expande para (1, altura, largura, 1)

# Prever probabilidades para o áudio processado
probabilidades = model.predict(mfccs)

# Converta a saída para um vetor (uma única amostra)
probabilidades = probabilidades.flatten()

# Exibir a probabilidade para cada palavra
for i, prob in enumerate(probabilidades):
    print(f"{palavras_desejadas[i]}: {prob * 100:.2f}%")

# Comando mais provável
predicted_command = palavras_desejadas[np.argmax(probabilidades)]
print(f"Comando previsto: {predicted_command}")